<a href="https://colab.research.google.com/github/EsmaeilNarimissa/YouTube-DataDriven-Upgrade/blob/main/DataExtrac_Youtube_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Data Extraction**

In [ ]:
import pandas as pd  # Import pandas for data manipulation
import matplotlib.pyplot as plt  # Import matplotlib for plotting
import seaborn as sns  # Import seaborn for enhanced data visualization
from googleapiclient.discovery import build  # Import build to create a YouTube API client
from googleapiclient.errors import HttpError  # Import HttpError for handling API errors
import traceback  # Import traceback for error logging
from isodate import parse_duration  # Import parse_duration to handle ISO 8601 durations
import time  # Import time for handling delays
from functools import wraps  # Import wraps for creating decorators

API_KEY = 'Use-Your-YoutTube-API-Key'  # YouTube Data API key

# Create a YouTube API client
youtube = build('youtube', 'v3', developerKey=API_KEY)

## 1.1 Exponential Backoff Retry Decorator

This code snippet defines a decorator, `retry_with_exponential_backoff`, which enhances a function with retry capabilities using exponential backoff.

- **Purpose**: To handle transient errors by retrying a function multiple times with increasing delays.
- **Functionality**:
  - **Retries**: Attempts to execute the decorated function up to five times (`max_retries`).
  - **Delay**: Starts with a 1-second delay (`retry_delay`) and doubles it after each failed attempt.
  - **Exception Handling**: If the function fails on the last attempt, the exception is raised.
- **Use Case**: Ideal for functions that may encounter temporary issues, such as network requests, allowing time for recovery between retries.

In [ ]:
def retry_with_exponential_backoff(func):
    @wraps(func)  # Preserve the original function's metadata
    def wrapper(*args, **kwargs):
        max_retries = 5  # Maximum number of retry attempts
        retry_delay = 1  # Initial delay between retries in seconds
        for attempt in range(max_retries):
            try:
                return func(*args, **kwargs)  # Attempt to execute the function
            except Exception as e:
                if attempt == max_retries - 1:  # If last attempt, raise the exception
                    raise
                # Log the failure and retry after a delay
                print(f"Attempt {attempt + 1} failed: {str(e)}. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)  # Wait before retrying
                retry_delay *= 2  # Double the delay for exponential backoff
    return wrapper  # Return the wrapped function

## 1.2 Channel ID Retrieval Function

This function, `get_channel_id`, retrieves the channel ID for a YouTube channel using its username.

- **Decorator**: Utilizes `retry_with_exponential_backoff` to handle transient errors by retrying the API call.
- **Parameters**:
  - `youtube`: YouTube API client object.
  - `username`: The username of the YouTube channel.
- **Process**:
  - Sends a search request to the YouTube API for the specified username.
  - Checks if the response contains channel data.
  - Returns the channel ID if found; otherwise, logs a message and returns `None`.
- **Error Handling**: Catches exceptions during the API call and logs an error message, returning `None` if an error occurs.

In [ ]:
@retry_with_exponential_backoff
def get_channel_id(youtube, username):
    try:
        request = youtube.search().list(
            part='snippet',  # Request the snippet part of the channel
            q=username,  # Query using the channel username
            type='channel',  # Specify the type as channel
            maxResults=1  # Limit the results to 1
        )
        response = request.execute()  # Execute the API request
        if 'items' in response and len(response['items']) > 0:
            return response['items'][0]['snippet']['channelId']  # Return the channel ID if found
        else:
            print(f"No channel found for username: {username}")
            return None  # Return None if no channel is found
    except Exception as e:
        print(f"An error occurred while getting channel ID: {e}")
        return None  # Return None if an error occurs

## 1.3 Channel Statistics Retrieval Function

This function retrieves statistics and information for a YouTube channel using the YouTube Data API.

- **Decorator**: Uses `retry_with_exponential_backoff` to manage transient errors by retrying the API call.
- **Parameters**:
  - `youtube`: YouTube API client object.
  - `channel_id`: ID of the YouTube channel.
- **Process**:
  - Sends a request to the YouTube API for channel data.
  - Executes the request and retrieves the response.
  - Checks if the response contains channel data.
  - Returns the response if successful; otherwise, logs a message and returns `None`.
- **Error Handling**: Catches `HttpError` exceptions, logs the error message, and returns `None` if an error occurs.

In [ ]:
@retry_with_exponential_backoff
def get_channel_stats(youtube, channel_id):
    try:
        request = youtube.channels().list(
            part="snippet,contentDetails,statistics",  # Request channel details
            id=channel_id  # Specify the channel ID
        )
        response = request.execute()  # Execute the API request

        if not response['items']:
            print(f"No channel found for ID: {channel_id}")
            return None  # Return None if no channel data is found

        return response['items'][0]  # Return the first (and only) item in the response
    except HttpError as e:
        print(f"An HTTP error {e.resp.status} occurred: {e.content}")
        return None  # Return None if an HTTP error occurs

## 1.4 Video ID and Details Retrieval Functions

These functions retrieve video IDs from a YouTube playlist and detailed information for each video.

`get_video_ids` Function
- **Decorator**: Uses `retry_with_exponential_backoff` to handle transient errors.
- **Parameters**:
  - `youtube`: YouTube API client object.
  - `playlist_id`: ID of the YouTube playlist.
- **Process**:
  - Sends a request to list items in the playlist.
  - Iterates through the response to collect video IDs.
  - Handles pagination to retrieve all video IDs in the playlist.
- **Error Handling**: Catches `HttpError` exceptions and logs the error message.

`get_video_details` Function
- **Decorator**: Uses `retry_with_exponential_backoff` to handle transient errors.
- **Parameters**:
  - `youtube`: YouTube API client object.
  - `video_ids`: List of YouTube video IDs.
- **Process**:
  - Batches video IDs in groups of 50 to comply with API limits.
  - Sends requests to retrieve detailed video information.
  - Collects and aggregates the video details from the API responses.
- **Error Handling**: Catches `HttpError` exceptions and logs the error message.

In [ ]:
@retry_with_exponential_backoff
def get_video_ids(youtube, playlist_id):
    video_ids = []
    request = youtube.playlistItems().list(
        part='contentDetails',  # Request content details of playlist items
        playlistId=playlist_id,  # Specify the playlist ID
        maxResults=50  # Limit the results to 50 per request
    )
    while request:
        try:
            response = request.execute()  # Execute the API request
            for item in response['items']:
                video_ids.append(item['contentDetails']['videoId'])  # Collect video IDs
            request = youtube.playlistItems().list_next(request, response)  # Get next page
        except HttpError as e:
            print(f"An HTTP error {e.resp.status} occurred: {e.content}")
            break  # Exit loop on error
    return video_ids  # Return the list of video IDs

@retry_with_exponential_backoff
def get_video_details(youtube, video_ids):
    all_video_info = []
    for i in range(0, len(video_ids), 50):
        try:
            request = youtube.videos().list(
                part='snippet,statistics,contentDetails,status,player,topicDetails',  # Request detailed video info
                id=','.join(video_ids[i:i+50])  # Batch video IDs in groups of 50
            )
            response = request.execute()  # Execute the API request
            all_video_info.extend(response['items'])  # Collect video details
        except HttpError as e:
            print(f"An HTTP error {e.resp.status} occurred: {e.content}")
    return all_video_info  # Return the list of video details

## 1.5 Video Data Parsing Function

This function, `parse_video_data`, processes detailed video information into a structured pandas DataFrame.

- **Parameters**:
  - `video_details`: List of video details retrieved from the YouTube API.
- **Process**:
  - Iterates over each video detail.
  - Extracts relevant information: video ID, title, description, published date, thumbnails, tags, category, language, duration, definition, caption, licensed content, projection, views, likes, comments, favorites, privacy status, license, embed HTML, and topic categories.
  - Appends the extracted data into a list of dictionaries.
- **Output**: Returns a pandas DataFrame containing the parsed video data, facilitating further analysis or storage.

In [ ]:
def parse_video_data(video_details):
    video_data = []
    for video in video_details:
        video_id = video['id']
        snippet = video['snippet']
        statistics = video['statistics']
        content_details = video['contentDetails']
        status = video['status']
        player = video.get('player', {})
        topic_details = video.get('topicDetails', {})

        # Convert publishedAt to datetime
        published_at = pd.to_datetime(snippet['publishedAt'])

        video_data.append({
            'video_id': video_id,
            'title': snippet['title'],
            'description': snippet.get('description', ''),
            'published_at': published_at,
            'thumbnails': snippet['thumbnails'],
            'tags': snippet.get('tags', []),
            'category_id': snippet['categoryId'],
            'default_language': snippet.get('defaultLanguage', ''),
            'default_audio_language': snippet.get('defaultAudioLanguage', ''),
            'duration': content_details['duration'],
            'definition': content_details['definition'],
            'caption': content_details['caption'],
            'licensed_content': content_details['licensedContent'],
            'projection': content_details['projection'],
            'views': int(statistics.get('viewCount', 0)),
            'likes': int(statistics.get('likeCount', 0)),
            'comments': int(statistics.get('commentCount', 0)),
            'favorites': int(statistics.get('favoriteCount', 0)),
            'privacy_status': status['privacyStatus'],
            'license': status['license'],
            'embed_html': player.get('embedHtml', ''),
            'topic_categories': topic_details.get('topicCategories', [])
        })

    df = pd.DataFrame(video_data)

    # Ensure 'published_at' is in datetime format
    df['published_at'] = pd.to_datetime(df['published_at'])

    return df

## 1.6 Custom Duration Parsing Function

This function, `parse_custom_duration`, converts ISO 8601 duration strings into a standard 'HH:MM:SS' format.

- **Parameters**:
  - `duration_str`: A string representing the duration in ISO 8601 format.
- **Process**:
  - Checks if the input is NaN or empty and returns a default duration of '00:00:00'.
  - Parses the duration string to calculate total seconds.
  - Converts total seconds into hours, minutes, and seconds.
  - Formats the result as 'HH:MM:SS'.
- **Error Handling**: Logs an error message and returns '00:00:00' if parsing fails.

In [ ]:
def parse_custom_duration(duration_str):
    if pd.isna(duration_str) or duration_str == '':
        return '00:00:00'  # Return default duration if input is NaN or empty

    try:
        duration = parse_duration(duration_str)  # Parse ISO 8601 duration
        total_seconds = int(duration.total_seconds())  # Convert duration to total seconds
        hours, remainder = divmod(total_seconds, 3600)  # Calculate hours and remainder
        minutes, seconds = divmod(remainder, 60)  # Calculate minutes and seconds
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}"  # Format as HH:MM:SS
    except:
        print(f"Error parsing duration: {duration_str}")  # Log parsing error
        return '00:00:00'  # Return default duration on error

## 1.7 Channel Data Extraction and Processing

This code snippet defines a function, `extract_channel_data`, to extract and process YouTube channel data, saving it to CSV files.

- **Decorator**: Uses `retry_with_exponential_backoff` to handle transient errors during API calls.
- **Process**:
  - Retrieves the channel ID using `get_channel_id`.
  - Fetches channel statistics with `get_channel_stats`.
  - Obtains the uploads playlist ID from the channel statistics.
  - Collects video IDs from the playlist using `get_video_ids`.
  - Retrieves detailed video information with `get_video_details`.
  - Parses the video data into a pandas DataFrame using `parse_video_data`.
  - Cleans and formats the data, converting dates and durations.
  - Saves the DataFrame to a CSV file named after the channel username.
- **Sequential Processing**: Iterates over a list of channel usernames, processing each one sequentially and storing results.
- **Error Handling**: Catches exceptions, logs errors, and continues processing other channels.

In [ ]:
@retry_with_exponential_backoff
def extract_channel_data(youtube, channel_username):
    try:
        print(f"Processing {channel_username}...")

        # Get channel ID
        channel_id = get_channel_id(youtube, channel_username)
        print(f"Channel ID for {channel_username}: {channel_id}")

        # Get channel stats
        channel_stats = get_channel_stats(youtube, channel_id)
        print(f"Channel stats retrieved for {channel_username}")
        print(f"Channel stats keys: {channel_stats.keys()}")

        # Get playlist ID for uploads
        uploads_playlist_id = channel_stats['contentDetails']['relatedPlaylists']['uploads']
        print(f"Uploads playlist ID for {channel_username}: {uploads_playlist_id}")

        # Get video IDs
        video_ids = get_video_ids(youtube, uploads_playlist_id)
        print(f"Number of videos found for {channel_username}: {len(video_ids)}")

        # Get video details
        video_details = get_video_details(youtube, video_ids)
        print(f"Video details retrieved for {channel_username}")

        # Parse video data
        df = parse_video_data(video_details)
        print(f"Data parsed for {channel_username}")
        print(f"DataFrame shape: {df.shape}")
        print(f"DataFrame columns: {df.columns}")
        print(f"Published_at dtype: {df['published_at'].dtype}")

        # Save to CSV
        csv_filename = f"{channel_username}_youtube_videos.csv"
        df['published_at'] = df['published_at'].dt.strftime('%Y-%m-%d')  # Convert to string in YYYY-MM-DD format
        df.to_csv(csv_filename, index=False)
        print(f"Data saved to {csv_filename}")

        return df
    except Exception as e:
        print(f"An error occurred while processing {channel_username}: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return None

# List of channel usernames
channels = [
    'canrevagency',      # Canada Revenue Agency
    'hmrcgovuk',         # UK Tax Office
    'ServicesAustralia', # Services Australia
    'healthgovau',       # Health.gov.au
    'AusTaxOffice',      # Australian Tax Office
    'InlandRevenueNZ'    # New Zealand IRD
]

# Sequential processing
results = []
for channel in channels:
    print(f"Processing {channel}...")
    try:
        df = extract_channel_data(youtube, channel)
        if df is not None and not df.empty:
            results.append((channel, df))
            print(f"Data extracted and saved for {channel}")
        else:
            print(f"Failed to extract data for {channel}")
    except Exception as e:
        print(f"An error occurred while processing {channel}: {str(e)}")
        import traceback
        print(traceback.format_exc())
    print("-----------------------------------")

print("All channels processed. Check the generated CSV files and results for further analysis.")

# Print final results
for channel, df in results:
    print(f"{channel}: {df.shape[0]} videos processed")

# Print any channels that failed
failed_channels = set(channels) - set(channel for channel, _ in results)
if failed_channels:
    print("Channels that failed:")
    for channel in failed_channels:
        print(f"- {channel}")

Processing canrevagency...
Processing canrevagency...
Channel ID for canrevagency: UCzcAdl16xQ5xnu0bg11QHQA
Channel stats retrieved for canrevagency
Channel stats keys: dict_keys(['kind', 'etag', 'id', 'snippet', 'contentDetails', 'statistics'])
Uploads playlist ID for canrevagency: UUzcAdl16xQ5xnu0bg11QHQA
Number of videos found for canrevagency: 169
Video details retrieved for canrevagency
Data parsed for canrevagency
DataFrame shape: (169, 22)
DataFrame columns: Index(['video_id', 'title', 'description', 'published_at', 'thumbnails',
       'tags', 'category_id', 'default_language', 'default_audio_language',
       'duration', 'definition', 'caption', 'licensed_content', 'projection',
       'views', 'likes', 'comments', 'favorites', 'privacy_status', 'license',
       'embed_html', 'topic_categories'],
      dtype='object')
Published_at dtype: datetime64[ns, UTC]
Data saved to canrevagency_youtube_videos.csv
Data extracted and saved for canrevagency
--------------------------------

The output indicates that the script successfully processed YouTube data for five channels. Here's a concise interpretation:

- **canrevagency**:
  - Channel ID: `UCzcAdl16xQ5xnu0bg11QHQA`
  - 169 videos processed and saved to `canrevagency_youtube_videos.csv`.

- **hmrcgovuk**:
  - Channel ID: `UCX03s8aGilPqvp992meZd6w`
  - 314 videos processed and saved to `hmrcgovuk_youtube_videos.csv`.

- **ServicesAustralia**:
  - Channel ID: `UCg5fHRiVL9fvAgQ-OP2FyHw`
  - 192 videos processed and saved to `ServicesAustralia_youtube_videos.csv`.

- **healthgovau**:
  - Channel ID: `UCf3BkFTwI4G1vMc6W-NpQNw`
  - 2200 videos processed and saved to `healthgovau_youtube_videos.csv`.

- **AusTaxOffice**:
  - Channel ID: `UCnFDRHQwcJm0YtIVxHeWJPw`
  - 50 videos processed and saved to `AusTaxOffice_youtube_videos.csv`.

Each channel's data was retrieved, parsed, cleaned, formatted, and saved to a corresponding CSV file. The process completed successfully for all channels, and the results are ready for further analysis.

## 1.8 CSV Data Integrity Checker



### 1.8.1
This script checks the integrity of CSV files containing YouTube video data.

- **File Handling**:
  - Uses `glob` to find all CSV files matching the pattern `*_youtube_videos.csv` in the current directory.
  - Verifies the existence of each file before processing.

- **Integrity Checks**:
  - **Row Count**: Prints the number of rows; warns if the file is empty.
  - **Column Verification**:
    - Checks for missing or extra columns against a predefined list of expected columns.
  - **Data Quality**:
    - Reports the number of null values in each column.
    - Checks for duplicate `video_id` entries.
    - Validates the date format in the `published_at` column.
    - Ensures numeric columns (`views`, `likes`, `comments`, `favorites`) contain numeric data.

- **Error Handling**: Catches and logs errors during file reading or processing.

- **Output**: Provides detailed feedback on the integrity of each CSV file, helping identify potential data issues.

In [ ]:
import pandas as pd
import os
import glob

def check_csv_integrity(file_path):
    print(f"Checking file: {file_path}")

    # Check if file exists
    if not os.path.exists(file_path):
        print(f"Error: File {file_path} does not exist.")
        return False

    try:
        # Try to read the CSV file
        df = pd.read_csv(file_path)

        # Check number of rows
        num_rows = len(df)
        print(f"Number of rows: {num_rows}")

        if num_rows == 0:
            print("Warning: CSV file is empty.")
            return False

        # Expected columns
        expected_columns = [
            'video_id', 'title', 'description', 'published_at', 'thumbnails',
            'tags', 'category_id', 'default_language', 'default_audio_language',
            'duration', 'definition', 'caption', 'licensed_content', 'projection',
            'views', 'likes', 'comments', 'favorites', 'privacy_status',
            'license', 'embed_html', 'topic_categories'
        ]

        # Check for missing columns
        missing_columns = set(expected_columns) - set(df.columns)
        if missing_columns:
            print(f"Warning: Missing columns: {missing_columns}")

        # Check for extra columns
        extra_columns = set(df.columns) - set(expected_columns)
        if extra_columns:
            print(f"Info: Extra columns found: {extra_columns}")

        # Basic data quality checks
        print("\nBasic data quality checks:")
        print(f"Null values:\n{df.isnull().sum()}")
        print(f"\nDuplicate video_ids: {df['video_id'].duplicated().sum()}")

        # Check date format
        if pd.api.types.is_datetime64_any_dtype(df['published_at']):
            print("Date format is valid (already in datetime format).")
        else:
            try:
                pd.to_datetime(df['published_at'], format='%Y-%m-%d')
                print("Date format is valid.")
            except ValueError:
                print("Warning: Invalid date format in 'published_at' column.")

        # Check numeric columns
        numeric_columns = ['views', 'likes', 'comments', 'favorites']
        for col in numeric_columns:
            if not pd.api.types.is_numeric_dtype(df[col]):
                print(f"Warning: Column '{col}' is not numeric.")

        print("\nData integrity check completed.")
        return True

    except Exception as e:
        print(f"Error reading or processing the CSV file: {str(e)}")
        return False

# Get all CSV files in the current directory
csv_files = glob.glob('*_youtube_videos.csv')

if not csv_files:
    print("No CSV files found in the current directory.")
else:
    for csv_file in csv_files:
        print("\n" + "="*50)
        check_csv_integrity(csv_file)
        print("="*50 + "\n")

print("Data integrity check completed for all files.")


Checking file: AusTaxOffice_youtube_videos.csv
Number of rows: 50

Basic data quality checks:
Null values:
video_id                   0
title                      0
description                0
published_at               0
thumbnails                 0
tags                       0
category_id                0
default_language          38
default_audio_language     8
duration                   0
definition                 0
caption                    0
licensed_content           0
projection                 0
views                      0
likes                      0
comments                   0
favorites                  0
privacy_status             0
license                    0
embed_html                 0
topic_categories           0
dtype: int64

Duplicate video_ids: 0
Date format is valid.

Data integrity check completed.


Checking file: canrevagency_youtube_videos.csv
Number of rows: 169

Basic data quality checks:
Null values:
video_id                    0
title                 

- **General Findings**:
  - All files have valid date formats and no duplicate video IDs.
  - Numeric columns ('views', 'likes', 'comments', 'favorites') are correctly formatted.

- **Specific Issues**:
  - **AusTaxOffice**: 38 missing 'default_language' and 8 missing 'default_audio_language'.
  - **canrevagency**: 2 missing 'description' and 158 missing 'default_language'.
  - **healthgovau**: 27 missing 'description' and 152 missing 'default_language'.
  - **hmrcgovuk**: 1 missing 'description', 303 missing 'default_language', and 1 missing 'default_audio_language'.
  - **InlandRevenueNZ**: 3 missing 'description', 56 missing 'default_language', and 1 missing 'default_audio_language'.
  - **ServicesAustralia**: 3 missing 'description', 151 missing 'default_language', and 4 missing 'default_audio_language'.

- **Conclusion**:
  - The primary data quality issue across all files is missing values in 'default_language' and, to a lesser extent, 'default_audio_language' and 'description'. These should be addressed based on the importance of these fields for your analysis.

### 1.8.2
The `comprehensive_integrity_check` function evaluates the integrity of a CSV file by performing the following checks:

1. **Read CSV File**:
   - Loads the file into a DataFrame.

2. **Initialize Issues List**:
   - Prepares to collect any data issues.

3. **Check for Missing Values**:
   - Identifies columns with missing values and records them.

4. **Check Data Types**:
   - Ensures 'views', 'likes', 'comments', and 'favorites' are numeric.

5. **Check for Duplicates**:
   - Detects duplicate entries in 'video_id'.

6. **Check Date Format**:
   - Validates the 'published_at' column is in datetime format.

7. **Check for Empty Strings**:
   - Finds columns with empty string values.

8. **Return Issues**:
   - Outputs a list of identified issues for each file.

This function is applied to multiple CSV files, reporting any integrity issues found.

In [ ]:
def comprehensive_integrity_check(filename):
    df = pd.read_csv(filename)

    # Initialize issues list
    issues = []

    # 1. Check for missing values
    missing_values = df.isnull().sum()
    if missing_values.any():
        issues.append(f"Missing values found in columns: {missing_values[missing_values > 0].to_dict()}")

    # 2. Check data types
    expected_numeric = ['views', 'likes', 'comments', 'favorites']
    for col in expected_numeric:
        if col in df.columns and not pd.api.types.is_numeric_dtype(df[col]):
            issues.append(f"Column {col} is not numeric")

    # 3. Check for duplicates
    if df['video_id'].duplicated().any():
        issues.append(f"Found {df['video_id'].duplicated().sum()} duplicate video IDs")

    # 4. Check date format
    if pd.api.types.is_datetime64_any_dtype(df['published_at']):
        print("Date format is valid (already in datetime format).")
    else:
        try:
            pd.to_datetime(df['published_at'], format='%Y-%m-%d')
        except ValueError:
            issues.append("Invalid date format in published_at column")

    # 5. Check for empty strings
    empty_strings = df.apply(lambda x: (x.astype(str).str.strip() == '').sum())
    if empty_strings.any():
        issues.append(f"Empty strings found in columns: {empty_strings[empty_strings > 0].to_dict()}")

    return issues

# Run checks for all files
for channel in channels:
    filename = f"{channel}_youtube_videos.csv"
    print(f"\nIntegrity check for {filename}:")
    issues = comprehensive_integrity_check(filename)

    if issues:
        print("Issues found:")
        for issue in issues:
            print(f"- {issue}")
    else:
        print("✅ No integrity issues found")


Integrity check for canrevagency_youtube_videos.csv:
Issues found:
- Missing values found in columns: {'description': 2, 'default_language': 158}

Integrity check for hmrcgovuk_youtube_videos.csv:
Issues found:
- Missing values found in columns: {'description': 1, 'default_language': 303, 'default_audio_language': 1}

Integrity check for ServicesAustralia_youtube_videos.csv:
Issues found:
- Missing values found in columns: {'description': 3, 'default_language': 151, 'default_audio_language': 4}

Integrity check for healthgovau_youtube_videos.csv:
Issues found:
- Missing values found in columns: {'description': 27, 'default_language': 152}

Integrity check for AusTaxOffice_youtube_videos.csv:
Issues found:
- Missing values found in columns: {'default_language': 38, 'default_audio_language': 8}

Integrity check for InlandRevenueNZ_youtube_videos.csv:
Issues found:
- Missing values found in columns: {'description': 3, 'default_language': 56, 'default_audio_language': 1}


- **canrevagency_youtube_videos.csv**:
  - 2 missing 'description' values.
  - 158 missing 'default_language' values.

- **hmrcgovuk_youtube_videos.csv**:
  - 1 missing 'description' value.
  - 303 missing 'default_language' values.
  - 1 missing 'default_audio_language' value.

- **ServicesAustralia_youtube_videos.csv**:
  - 3 missing 'description' values.
  - 151 missing 'default_language' values.
  - 4 missing 'default_audio_language' values.

- **healthgovau_youtube_videos.csv**:
  - 27 missing 'description' values.
  - 152 missing 'default_language' values.

- **AusTaxOffice_youtube_videos.csv**:
  - 38 missing 'default_language' values.
  - 8 missing 'default_audio_language' values.

**Summary**: The primary issue across all files is missing values in 'default_language', with some files also missing 'description' and 'default_audio_language' values.

## 1.9 Combining Data into a CSV

In [ ]:
import pandas as pd

# Updated list of channel names
channels = [
    'canrevagency',
    'hmrcgovuk',
    'ServicesAustralia',
    'healthgovau',
    'AusTaxOffice',
    'InlandRevenueNZ'  # Added New Zealand IRD
]

# Updated dictionary with NZ tax agency
channel_info = {
    'canrevagency': {'country': 'CA', 'domain': 'TAX'},
    'hmrcgovuk': {'country': 'UK', 'domain': 'TAX'},
    'ServicesAustralia': {'country': 'AUS', 'domain': 'SERVICE'},
    'healthgovau': {'country': 'AUS', 'domain': 'HEALTH'},
    'AusTaxOffice': {'country': 'AUS', 'domain': 'TAX'},
    'InlandRevenueNZ': {'country': 'NZ', 'domain': 'TAX'}  # Added NZ mapping
}

# Load all CSV files into a list of dataframes
dfs = []
for channel in channels:
    filename = f"{channel}_youtube_videos.csv"
    df = pd.read_csv(filename)
    df['channel'] = channel
    df['country'] = channel_info[channel]['country']
    df['domain'] = channel_info[channel]['domain']
    dfs.append(df)

# Concatenate all dataframes
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined dataframe to a CSV file
combined_df.to_csv('combined_channels.csv', index=False)

print("Combined CSV file created successfully.")
print(f"Total number of videos: {len(combined_df)}")
print("\nVideos by Channel, Country, and Domain:")
print(combined_df[['channel', 'country', 'domain']].value_counts())

Combined CSV file created successfully.
Total number of videos: 2993

Videos by Channel, Country, and Domain:
channel            country  domain 
healthgovau        AUS      HEALTH     2211
hmrcgovuk          UK       TAX         314
ServicesAustralia  AUS      SERVICE     192
canrevagency       CA       TAX         169
InlandRevenueNZ    NZ       TAX          57
AusTaxOffice       AUS      TAX          50
Name: count, dtype: int64


In [ ]:
import pandas as pd
import random
import warnings

# Suppress the specific DeprecationWarning
warnings.filterwarnings("ignore", category=DeprecationWarning, message=".*DataFrameGroupBy.apply operated on the grouping columns.*")

# Read the combined CSV file
df = pd.read_csv('combined_channels.csv')

# Get 2 random rows for each channel
sample_rows = df.groupby('channel').apply(lambda x: x.sample(n=2, random_state=random.randint(0, 1000))).reset_index(drop=True)

# Display the sampled rows
print(sample_rows.to_string())

# Optionally, you can save these sample rows to a new CSV file
# sample_rows.to_csv('sample_rows.csv', index=False)

       video_id                                                                  title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  